In [80]:
# import moduels
import pandas as pd
import numpy as np
import requests 
from bs4 import BeautifulSoup
import os
import pandas as pd
import numpy as np
import time # 걸린 시간 측정 
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

# %%writefile C:\Users\User\Documents\python_basic\paper_implementation\module_name.py
# Naver 콜금리는 지속적으로 업데이트 되기 때문에, 수집하고자 하는 범위를 항상 확인하고 실행할 것 
class Callrate_package(object): # 콜금리를 수집하는 Class 생성
    callrate_list = [] # callrate를 담을 빈 리스트 생성

    def Crawling_callrate(self, start, end): # start와 end는 page number
        self.start = start
        self.end = end
        
        for i in range(start, end): # for 문을 통해 page 접근 
            print(i,'번 째 페이지를 가져오고 있습니다...')
            url = 'https://finance.naver.com/marketindex/interestDailyQuote.nhn?marketindexCd=IRR_CALL&page={}'.format(i)

            response = requests.get(url)
            soup = BeautifulSoup(response.content, 'html.parser')
            content = soup.find('table', class_='tbl_exchange today').find_all('tr') # tag를 찾아 접근 

            for td in content:
                temp_dict = {} # 임시 dict를 생성하고, 후에 list에 append

                date = td.find('td', class_='date') # tag를 찾아 접근 
                if date is not None: # None 값들을 제외하고 수집
                    date = date.text.replace('\n','').replace('\t','') # 개행문자 \n, \t 삭제
                    temp_dict['date']= date

                callrate = td.find('td', class_='num')
                if callrate is not None:
                    callrate = callrate.text
                    temp_dict['callrate'] = float(callrate)

                    callrate_package.callrate_list.append(temp_dict)

    def Labeling_callrate(self, datelimit): # datelimit 은 수집하고자하는 마지막 날짜
        df = pd.DataFrame(callrate_package.callrate_list) # df 생성
        df['old_date'] = None # 한달전 date
        df['old_callrate'] = None # 한달전 콜금리
        df['updown'] = None # labeling
        
        df['date'] = pd.to_datetime(df['date']) # datetime 형식으로 변경
        df = df.set_index(df['date']) # datetime을 index로 변경 

        for i in range(len(df.index)): # 수집한 index를 기반으로 반복문 시작
            if df['date'][i] >= pd.Timestamp(datelimit): # 2005-01-01 까지 
                df['old_date'][i] = df['date'][i] - pd.DateOffset(months = 1)

                while df['old_date'][i] not in df['date']: 
                    df['old_date'][i] = df['old_date'][i] - pd.DateOffset(days=1) # 하루씩 빼면서 콜금리가 있는 날짜를 찾는다.

                # old_date의 index의 값을 가져온다.    
                if df['old_date'][i] in df.index:
                    df['old_callrate'][i] = df.loc[df['old_date'][i]]['callrate']

                # 한달전 콜금리와 비교해보았을 때,
                try:
                    if (df['callrate'][i] - df['old_callrate'][i]) < 0:
                        df['updown'][i] = 'down'
                    elif (df['callrate'][i] - df['old_callrate'][i]) == 0:
                        df['updown'][i] = 'neutrality'
                    else:
                        df['updown'][i] = 'up'
                except:
                    pass

        print(df)
        return df.to_csv("callrate.csv")
    
    def Check_callrate(self, datelimit):
        check = pd.read_csv('./callrate1.csv', index_col = 0)
        check.index = pd.to_datetime(check.index)
        check = check[check.index >= pd.Timestamp(datelimit)]
        print('Days: \n', len(check))
        print('\n')
        print('Columns: \n', check.columns)
        print('\n')
        print('Updown Valuecounts: \n',check['updown'].value_counts())
        print('\n')
        print(check.head())
        print(check.tail())
        check.to_csv('callrate_final.csv', encoding='cp949')

In [81]:
CR = Callrate_package()

In [82]:
CR.Crawling_callrate(1, 530)

1 번 째 페이지를 가져오고 있습니다...
2 번 째 페이지를 가져오고 있습니다...
3 번 째 페이지를 가져오고 있습니다...
4 번 째 페이지를 가져오고 있습니다...
5 번 째 페이지를 가져오고 있습니다...
6 번 째 페이지를 가져오고 있습니다...
7 번 째 페이지를 가져오고 있습니다...
8 번 째 페이지를 가져오고 있습니다...
9 번 째 페이지를 가져오고 있습니다...
10 번 째 페이지를 가져오고 있습니다...
11 번 째 페이지를 가져오고 있습니다...
12 번 째 페이지를 가져오고 있습니다...
13 번 째 페이지를 가져오고 있습니다...
14 번 째 페이지를 가져오고 있습니다...
15 번 째 페이지를 가져오고 있습니다...
16 번 째 페이지를 가져오고 있습니다...
17 번 째 페이지를 가져오고 있습니다...
18 번 째 페이지를 가져오고 있습니다...
19 번 째 페이지를 가져오고 있습니다...
20 번 째 페이지를 가져오고 있습니다...
21 번 째 페이지를 가져오고 있습니다...
22 번 째 페이지를 가져오고 있습니다...
23 번 째 페이지를 가져오고 있습니다...
24 번 째 페이지를 가져오고 있습니다...
25 번 째 페이지를 가져오고 있습니다...
26 번 째 페이지를 가져오고 있습니다...
27 번 째 페이지를 가져오고 있습니다...
28 번 째 페이지를 가져오고 있습니다...
29 번 째 페이지를 가져오고 있습니다...
30 번 째 페이지를 가져오고 있습니다...
31 번 째 페이지를 가져오고 있습니다...
32 번 째 페이지를 가져오고 있습니다...
33 번 째 페이지를 가져오고 있습니다...
34 번 째 페이지를 가져오고 있습니다...
35 번 째 페이지를 가져오고 있습니다...
36 번 째 페이지를 가져오고 있습니다...
37 번 째 페이지를 가져오고 있습니다...
38 번 째 페이지를 가져오고 있습니다...
39 번 째 페이지를 가져오고 있습니다...
40 번 째 페이지를 가져오고 있습니다...
41 번 째 페이

322 번 째 페이지를 가져오고 있습니다...
323 번 째 페이지를 가져오고 있습니다...
324 번 째 페이지를 가져오고 있습니다...
325 번 째 페이지를 가져오고 있습니다...
326 번 째 페이지를 가져오고 있습니다...
327 번 째 페이지를 가져오고 있습니다...
328 번 째 페이지를 가져오고 있습니다...
329 번 째 페이지를 가져오고 있습니다...
330 번 째 페이지를 가져오고 있습니다...
331 번 째 페이지를 가져오고 있습니다...
332 번 째 페이지를 가져오고 있습니다...
333 번 째 페이지를 가져오고 있습니다...
334 번 째 페이지를 가져오고 있습니다...
335 번 째 페이지를 가져오고 있습니다...
336 번 째 페이지를 가져오고 있습니다...
337 번 째 페이지를 가져오고 있습니다...
338 번 째 페이지를 가져오고 있습니다...
339 번 째 페이지를 가져오고 있습니다...
340 번 째 페이지를 가져오고 있습니다...
341 번 째 페이지를 가져오고 있습니다...
342 번 째 페이지를 가져오고 있습니다...
343 번 째 페이지를 가져오고 있습니다...
344 번 째 페이지를 가져오고 있습니다...
345 번 째 페이지를 가져오고 있습니다...
346 번 째 페이지를 가져오고 있습니다...
347 번 째 페이지를 가져오고 있습니다...
348 번 째 페이지를 가져오고 있습니다...
349 번 째 페이지를 가져오고 있습니다...
350 번 째 페이지를 가져오고 있습니다...
351 번 째 페이지를 가져오고 있습니다...
352 번 째 페이지를 가져오고 있습니다...
353 번 째 페이지를 가져오고 있습니다...
354 번 째 페이지를 가져오고 있습니다...
355 번 째 페이지를 가져오고 있습니다...
356 번 째 페이지를 가져오고 있습니다...
357 번 째 페이지를 가져오고 있습니다...
358 번 째 페이지를 가져오고 있습니다...
359 번 째 페이지를 가져오고 있습니다...
360 번 째 페이지를

In [83]:
CR.Labeling_callrate('2005-01-01T00')

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See th

            callrate       date             old_date old_callrate      updown
date                                                                         
2019-09-06      1.53 2019-09-06  2019-08-06 00:00:00         1.49          up
2019-09-05      1.50 2019-09-05  2019-08-05 00:00:00         1.49          up
2019-09-04      1.49 2019-09-04  2019-08-02 00:00:00         1.48          up
2019-09-03      1.50 2019-09-03  2019-08-02 00:00:00         1.48          up
2019-09-02      1.50 2019-09-02  2019-08-02 00:00:00         1.48          up
2019-08-30      1.54 2019-08-30  2019-07-30 00:00:00         1.52          up
2019-08-29      1.51 2019-08-29  2019-07-29 00:00:00          1.5          up
2019-08-28      1.51 2019-08-28  2019-07-26 00:00:00         1.51  neutrality
2019-08-27      1.53 2019-08-27  2019-07-26 00:00:00         1.51          up
2019-08-26      1.53 2019-08-26  2019-07-26 00:00:00         1.51          up
2019-08-23      1.53 2019-08-23  2019-07-23 00:00:00         1.4

In [84]:
CR.Check_callrate('2005-01-01')

Days: 
 3644


Columns: 
 Index(['callrate', 'date.1', 'old_date', 'old_callrate', 'updown'], dtype='object')


Updown Valuecounts: 
 down          1505
up            1478
neutrality     661
Name: updown, dtype: int64


            callrate      date.1             old_date  old_callrate updown
date                                                                      
2019-09-06      1.53  2019-09-06  2019-08-06 00:00:00          1.49     up
2019-09-05      1.50  2019-09-05  2019-08-05 00:00:00          1.49     up
2019-09-04      1.49  2019-09-04  2019-08-02 00:00:00          1.48     up
2019-09-03      1.50  2019-09-03  2019-08-02 00:00:00          1.48     up
2019-09-02      1.50  2019-09-02  2019-08-02 00:00:00          1.48     up
            callrate      date.1             old_date  old_callrate updown
date                                                                      
2005-01-07      3.22  2005-01-07  2004-12-07 00:00:00          3.19     up
2005-01-06      3.21  2005-01-